In [1]:

import torch.nn as nn

In [2]:
cfgs = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

In [4]:
class VGG(nn.Module):
    def __init__(self, cfg, batch_norm, num_classes=1000, init_weights=True):
        super().__init__()
        self.features = self.make_layers(cfg, batch_norm)
        self.avgpool = nn.AdaptiveAvgPool2d((7,7))
        # 마지막 Conv 레이어의 출력을 고정된 크기 (7×7)로 줄여주는 레이어
        self.classifier = nn.Sequential(
            nn.Linear(512*7*7, 4096),
            nn.ReLU(True),
            nn.Dropout(),

            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),

            nn.Linear(4096, num_classes))
        '''	•	마지막 출력은 보통 CrossEntropyLoss와 함께 사용됨
            •	이 Loss는 내부적으로 **LogSoftmax**를 포함하고 있어서
                → ReLU 같은 비선형 함수 쓰면 안 됨 (출력 왜곡됨)'''


        if init_weights:
            self._initialize_weigths()

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x,1)
        x = self.classifier(x)
        return x



        # 초기 가중치, 바이어스 초기화
    # 즉, 사실상 PyTorch는 이미 나쁘지 않은 기본 초기화를 제공하고 있어.
    # ❗ 그런데 왜 따로 초기화 함수를 쓰냐?
# 그래서 _initialize_weights()를 안 써도 “망하지는 않음”.
    # 	Kaiming He 초기화 (ReLU에 적합)
    # mode='fan_out': 출력 채널 수 기준으로 정규화
    # nonlinearity='relu': ReLU에 맞는 분산 설정
    def _initialize_weigths(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

            elif isinstance(m, nn.BatchNorm2d):
                # 가중치(γ)는 1, 편향(β)은 0으로 초기화 → 초기에는 입력 그대로 통과
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
            # 평균 0, 표준편차 0.01인 정규분포로 초기화 → 작은 값으로 안정적인 시작
            # 바이어스는 0
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

    # 'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],

# cfg 리스트를 읽으면서:
# 숫자 → Conv2d + (BatchNorm) + ReLU
# 	'M' → MaxPooling 레이어
# → 결과적으로 CNN의 features 블록을 구성하는 레이어들이 쭉 nn.Sequential로 반환됨.

    def make_layers(self, cfg, batch_norm=False):
        layers=[]
        in_channels = 3
        for v in cfg:
            if v == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
                if batch_norm:
                    layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(True)]
                else:
                    layers += [conv2d, nn.ReLU(True)]
                in_channels = v

        return nn.Sequential(*layers)


In [5]:

from torchinfo import summary

model_vgg16_bn = VGG(cfgs['D'], batch_norm=True)
summary(model_vgg16_bn, input_size=(2,3,224,224))

Layer (type:depth-idx)                   Output Shape              Param #
VGG                                      [2, 1000]                 --
├─Sequential: 1-1                        [2, 512, 7, 7]            --
│    └─Conv2d: 2-1                       [2, 64, 224, 224]         1,792
│    └─BatchNorm2d: 2-2                  [2, 64, 224, 224]         128
│    └─ReLU: 2-3                         [2, 64, 224, 224]         --
│    └─Conv2d: 2-4                       [2, 64, 224, 224]         36,928
│    └─BatchNorm2d: 2-5                  [2, 64, 224, 224]         128
│    └─ReLU: 2-6                         [2, 64, 224, 224]         --
│    └─MaxPool2d: 2-7                    [2, 64, 112, 112]         --
│    └─Conv2d: 2-8                       [2, 128, 112, 112]        73,856
│    └─BatchNorm2d: 2-9                  [2, 128, 112, 112]        256
│    └─ReLU: 2-10                        [2, 128, 112, 112]        --
│    └─Conv2d: 2-11                      [2, 128, 112, 112]        147,

In [6]:
model_vgg19_bn = VGG(cfgs['E'], batch_norm=True)
summary(model_vgg19_bn, input_size=(2,3,224,224))

Layer (type:depth-idx)                   Output Shape              Param #
VGG                                      [2, 1000]                 --
├─Sequential: 1-1                        [2, 512, 7, 7]            --
│    └─Conv2d: 2-1                       [2, 64, 224, 224]         1,792
│    └─BatchNorm2d: 2-2                  [2, 64, 224, 224]         128
│    └─ReLU: 2-3                         [2, 64, 224, 224]         --
│    └─Conv2d: 2-4                       [2, 64, 224, 224]         36,928
│    └─BatchNorm2d: 2-5                  [2, 64, 224, 224]         128
│    └─ReLU: 2-6                         [2, 64, 224, 224]         --
│    └─MaxPool2d: 2-7                    [2, 64, 112, 112]         --
│    └─Conv2d: 2-8                       [2, 128, 112, 112]        73,856
│    └─BatchNorm2d: 2-9                  [2, 128, 112, 112]        256
│    └─ReLU: 2-10                        [2, 128, 112, 112]        --
│    └─Conv2d: 2-11                      [2, 128, 112, 112]        147,

In [7]:

import torch
import torch.nn as nn

데이터 전처리
- 이미지 변환은 이미지 데이터의 크기를 256으로 키웠다가 224로 중앙 자르기 수행
- 탐지하려는 객체가 중앙에 위치할 확률이 높으므로 불필요한 지역특징을 제거하기 위한 전처리 방법

- 입력 이미지 크기를 224로 바로 리사이즈 할 수도 있지만, 그러면 검출하려는 객체의 크기가 더 작아질 수 있음

일반적으로 **사전학습(pretrained) 모델**을 사용할 때는 ImageNet으로 학습된 모델의 입력 분포에 맞추기 위해 같은 정규화 값(평균, 표준편차)을 사용합니다.

- **사전학습 모델 활용 시**:
  - ImageNet으로 학습된 모델은 입력 이미지가 이 정규화 값을 기준으로 전처리되었을 때 최적의 성능을 보입니다.
  - 따라서 새로운 데이터셋이 ImageNet과 유사한 분포(자연 이미지 등)를 가진다면 같은 값을 사용하는 것이 좋습니다.

- **새로운 데이터셋이 ImageNet과 많이 다를 경우**:
  - 만약 데이터셋의 특성이 크게 다르다면(예: 의료 영상, 위성 사진 등)
  - 새로운 데이터셋 자체의 평균과 표준편차를 계산하여 사용하는 것이 더 적합할 수 있습니다.

In [10]:
device = torch.device('mps') # cpu, mps
print(device)

mps


In [ ]:
from collections import defaultdict



# 라벨이 0이면 고양이, 1이면 강아지
def lable_to_str(lable):
    return "Cat" if lable == 0 else "Dog"

def visualize_by_category(dataset, n_samples = 5, cmap = None):
    samples = defaultdict(list)
    for img, label in dataset:
        if len(samples[label]) < n_samples:
            samples[label].append(img)





In [4]:
sap = {'a':123, 'b':34234}
for k,v in sap.items():
    print(k,v)
for k in sap:
    print(k, sap[k])

a 123
b 34234
a 123
b 34234
